In [44]:
import numpy as np
import pandas as pd
import csv
import xgboost
print(xgboost.__version__)
from matplotlib import pyplot
import xgboost as xgb
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from numpy import absolute
from pandas import read_csv
from sklearn.model_selection import train_test_split

from sklearn.decomposition import PCA

from sklearn.model_selection import RandomizedSearchCV,GridSearchCV

1.3.3


In [45]:
test_df = pd.read_csv('test.csv')
train_df = pd.read_csv('train.csv')

test_df.describe()
if train_df.isna().sum().sum() !=0:
    train_df = train_df.fillna(train_df.mean())

In [57]:

N=5

y = train_df.loss
X = train_df.drop(columns=['loss'])

pca = PCA(n_components=N)
principalComponents = pca.fit_transform(X)
X = pd.DataFrame(data = principalComponents, columns = [('P' + str(i)) for i in range(N)])

principalComponents = pca.fit_transform(test_df)
test_df = pd.DataFrame(data = principalComponents, columns = [('P' + str(i)) for i in range(N)])

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.9, random_state=42)
X_train = X
y_train = y

print(X_train.shape)
# print(X_test.shape)
print(y_train.shape)
# print(y_test.shape)

print(test_df.shape)


(250000, 5)
(250000,)
(150000, 5)


In [ ]:

model = xgb.XGBRegressor(subsample=0.7)

# reg_cv = GridSearchCV(model, {"colsample_bytree":[0.8],"min_child_weight":[1.0]
#                             ,'max_depth': [6], 'n_estimators':[1000], 'eta':[0.1]}, verbose=1)
reg_cv = GridSearchCV(model, {"colsample_bytree":[1.0, 0.8],"min_child_weight":[1.0]
                            ,'max_depth': [6, 8, 10], 'n_estimators':[800, 1000, 1200], 'eta':[0.1, 0.2]}, verbose=1)
reg_cv.fit(X_train,y_train)
reg_cv.best_params_

Fitting 5 folds for each of 36 candidates, totalling 180 fits


In [ ]:
gbm = xgb.XGBRegressor(**reg_cv.best_params_)
# gbm = xgb.XGBRegressor(n_estimators=1000, max_depth=7, eta=0.1, subsample=0.7, colsample_bytree=0.8)
gbm.fit(X_train,y_train)

predictions = gbm.predict(test_df)
print(predictions)

# print(gbm.score(X_test,y_test))
print(gbm.score(X_train,y_train))

In [ ]:
submission = pd.read_csv('sample_submission.csv')
submission[submission.columns[1:]] = predictions
submission.to_csv('submission.csv', index=False)
